<a href="https://colab.research.google.com/github/syh0397/Statistics_python/blob/main/7_Anova_test(Two_way).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pingouin
!pip install --upgrade pingouin
!pip install scikit_posthocs

## 이원배치분산분석(Two-Way ANOVA)(Analysis of Variance)

#### 설명

- 일원배치 = 독립변수가 하나 
- 이원배치 =  독립변수 두개 

    - 두개의 범주형 독립변수가  하나의 연속형 종속변수에 미치는 영향을 본다 .   평균차이를 본다.  

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd 
import numpy as np 

import os
import sys
import warnings
warnings.filterwarnings('ignore')

import sklearn 
import missingno as msno
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('seaborn')
sns.set(font_scale=2.5)

import scipy 
from scipy import  stats
import statsmodels.api as sm
import pylab as py

import pingouin as pg
import scikit_posthocs

pd.options.display.float_format = '{:.3f}'.format

In [10]:
sns.get_dataset_names()

['anagrams',
 'anscombe',
 'attention',
 'brain_networks',
 'car_crashes',
 'diamonds',
 'dots',
 'exercise',
 'flights',
 'fmri',
 'gammas',
 'geyser',
 'iris',
 'mpg',
 'penguins',
 'planets',
 'taxis',
 'tips',
 'titanic']

In [11]:
a = sns.load_dataset('tips')
a

,total_bill,tip,sex,smoker,day,time,size
0,16.990,1.010,Female,No,Sun,Dinner,2
1,10.340,1.660,Male,No,Sun,Dinner,3
2,21.010,3.500,Male,No,Sun,Dinner,3
3,23.680,3.310,Male,No,Sun,Dinner,2
4,24.590,3.610,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.030,5.920,Male,No,Sat,Dinner,3
240,27.180,2.000,Female,Yes,Sat,Dinner,2
241,22.670,2.000,Male,Yes,Sat,Dinner,2
242,17.820,1.750,Male,No,Sat,Dinner,2


- 이원배치분산분석에서는 두개의 상호작용도 볼 수 있다
- 아래에서 day 변수와  size 변수 두개의 상호작용에 대한 효과가 total_bill 의 평균 차이에 나타나는가? 

In [12]:
hos = a[['day','size']]
tip = a['tip']
hos['tip'] = tip
hos

,day,size,tip
0,Sun,2,1.010
1,Sun,3,1.660
2,Sun,3,3.500
3,Sun,2,3.310
4,Sun,4,3.610
...,...,...,...
239,Sat,3,5.920
240,Sat,2,2.000
241,Sat,2,2.000
242,Sat,2,1.750


In [13]:
print(pg.anova(dv = 'tip',
               between = ['day', 'size'],
               data = hos))

       Source      SS      DF      MS       F  p-unc   np2
0         day  26.437   3.000   8.812   5.862  0.001 0.073
1        size 995.013   5.000 199.003 132.371  0.000 0.747
2  day * size 111.290  15.000   7.419   4.935  0.000 0.248
3    Residual 336.754 224.000   1.503     NaN    NaN   NaN


- day와 size는 0.05보다 작다. (귀무가설 기각)
- tip의 평균차이가 집단별로 있다고 볼 수 있음.
- 또한 day 와 size 의 상호작용에 대한 p - value 값도 귀무가설을 기각한다. 즉, 상호작용에 대해서도 통계적으로 유의미 하다 (평균차이가 있다.) 

#### 사후검정

- 등분산이 가정되었을 때 Scheffe, Tukey 또는 Duncan을 많이 사용합니다.
- 등분산이 가정되지 않았을 때 Dunnett T3 또는 Games-Howell를 사용합니다.

- Scheffe
  - 특징 : 가장보수적이고 엄격한 사후검정 방식
  - 장점 : 엄격한 기준으로 사후 검정 실시
  - 단점 : 통계적으로 유의한 차이를 도출하기가 쉽지 않음


1. Duncan와 Tukey의 경우 변인의 수가 비슷할 때 사용합니다.
    - ex) 1학년= 35명, 2학년 =36명, 3학년 34명
 
2. Scheffe는 경우 변인의 수가 일정하지 않을 때 사용합니다.
    - ex) 1학년= 45명, 2학년 =26명, 3학년 64명

In [14]:
hos['day'] = hos['day'].astype(str)
print(scikit_posthocs.posthoc_scheffe(hos, val_col='tip', group_col='day'))

       Sun   Sat  Thur   Fri
Sun  1.000 0.690 0.243 0.539
Sat  0.690 1.000 0.817 0.908
Thur 0.243 0.817 1.000 1.000
Fri  0.539 0.908 1.000 1.000


In [15]:
hos['size'] = hos['size'].astype(str)
print(scikit_posthocs.posthoc_scheffe(hos, val_col='tip', group_col='size'))

      2     3     4     1     6     5
2 1.000 0.020 0.000 0.627 0.003 0.233
3 0.020 1.000 0.223 0.098 0.147 0.943
4 0.000 0.223 1.000 0.004 0.713 1.000
1 0.627 0.098 0.004 1.000 0.002 0.075
6 0.003 0.147 0.713 0.002 1.000 0.825
5 0.233 0.943 1.000 0.075 0.825 1.000
